In [1]:
import pandas as pd
import numpy as np
import glob

In [2]:
path ='C:/Users/sreekar chidurala/Desktop/july work/quantum black AV/drive' # use your path
all_files = glob.glob(path + "/*.parquet")

li = []

for filename in all_files:
    df = pd.read_parquet(filename)
    li.append(df)

drive_frame = pd.concat(li, axis=0, ignore_index=True)

C:\Users\sreekar chidurala\Anaconda3\envs\sreekarr\lib\site-packages\pyarrow\pandas_compat.py:708: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels = getattr(columns, 'labels', None) or [
C:\Users\sreekar chidurala\Anaconda3\envs\sreekarr\lib\site-packages\pyarrow\pandas_compat.py:735: FutureWarning: the 'labels' keyword is deprecated, use 'codes' instead
  return pd.MultiIndex(levels=new_levels, labels=labels, names=columns.names)
C:\Users\sreekar chidurala\Anaconda3\envs\sreekarr\lib\site-packages\pyarrow\pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


In [3]:
drive_frame.head()

,vehicle_id,trip_id,datetime,velocity,accel_x,accel_y,accel_z,engine_coolant_temp,eng_load,fuel_level,iat,rpm
0,1000515,db3c08b01205459294bbc91e3f7dfeb6,2017-01-06 23:00:00,0.00,50.0,65.13,87.92,150.70,192.88,118.66,95.0,2107.44
1,1000515,db3c08b01205459294bbc91e3f7dfeb6,2017-01-06 23:00:01,47.49,51.0,72.34,83.78,154.76,196.04,116.69,100.0,2119.17
2,1000515,db3c08b01205459294bbc91e3f7dfeb6,2017-01-06 23:00:02,57.94,50.0,69.20,87.73,165.50,196.01,105.46,90.0,2112.32
3,1000515,db3c08b01205459294bbc91e3f7dfeb6,2017-01-06 23:00:03,57.50,48.0,70.83,84.95,157.32,193.31,110.35,90.0,2092.94
4,1000515,db3c08b01205459294bbc91e3f7dfeb6,2017-01-06 23:00:04,52.29,47.0,70.22,92.90,151.00,189.49,104.25,97.0,2117.87


In [4]:
drive_frame.sort_values(by = ['vehicle_id','datetime'], inplace=True)

## Engine Features

### Torque Features Test

In [5]:
def torque_feat(df, lower=60):
    df = df.copy()
    df['torque'] = df['eng_load']/255
    m1 = (df['torque'] >= lower/100) & (df['torque'] < (lower + 10)/100)
    fname = 'ft_torque_util_' + str(lower) + 'pct_s'
    df[fname] = np.where(m1,1,0)
    df['datetime'] = pd.to_datetime(df['datetime']) - pd.to_timedelta(7, unit='h')
    df['Date'] = df['datetime'] - pd.to_timedelta(7, unit='d')
    df1 = df.groupby(['vehicle_id', pd.Grouper(key='Date', freq='W-SUN')])[fname].sum().reset_index().sort_values('Date')
    df1['week_start_date'] = df1['Date'] + pd.to_timedelta(1, unit='d')
    df1.drop(['Date'],axis = 1, inplace = True)
    cols = ['vehicle_id','week_start_date',fname]
    df1 = df1[cols]
    return df1

In [6]:
for i in [60,70,80,90]:
    temp_df = pd.DataFrame()
    if i == 60:
        fin_df = torque_feat(drive_frame, lower = i)
    else:
        temp_df = torque_feat(drive_frame, lower=i)
        fin_df = fin_df.merge(temp_df, how = 'left', on = ['vehicle_id','week_start_date'])

In [7]:
fin_df.head()

,vehicle_id,week_start_date,ft_torque_util_60pct_s,ft_torque_util_70pct_s,ft_torque_util_80pct_s,ft_torque_util_90pct_s
0,1000500,2016-12-26,0,6570,8909,9
1,1000504,2016-12-26,0,22722,21651,1
2,1000505,2016-12-26,0,74,9179,36
3,1000506,2016-12-26,74,8405,8054,0
4,1000507,2016-12-26,0,3517,10497,1


### HorsePower Features

In [10]:
vehicle_data = pd.read_csv("vehicle.csv", error_bad_lines=False)

b'Skipping line 8: expected 17 fields, saw 33\nSkipping line 15: expected 17 fields, saw 33\nSkipping line 22: expected 17 fields, saw 33\nSkipping line 29: expected 17 fields, saw 33\n'


In [11]:
merged_drive_frame = drive_frame.merge(vehicle_data[['vehicle_id','max_torque','max_horsepower','max_horsepower_rpm']], how='left', on = 'vehicle_id')

In [12]:
merged_drive_frame['horsepower_util'] = ((merged_drive_frame['eng_load']/255)*merged_drive_frame['max_torque']*merged_drive_frame['rpm'])/(5252*merged_drive_frame['max_horsepower'])

In [13]:
def horsepower_feat(df, lower=50):
    df = df.copy()
    m1 = (df['horsepower_util'] >= lower/100) & (df['horsepower_util'] < (lower + 10)/100)
    fname = 'ft_horsepower_util_' + str(lower) + 'pct_s'
    df[fname] = np.where(m1,1,0)
    df['datetime'] = pd.to_datetime(df['datetime']) - pd.to_timedelta(7, unit='h')
    df['Date'] = df['datetime'] - pd.to_timedelta(7, unit='d')
    df1 = df.groupby(['vehicle_id', pd.Grouper(key='Date', freq='W-SUN')])[fname].sum().reset_index().sort_values('Date')
    df1['week_start_date'] = df1['Date'] + pd.to_timedelta(1, unit='d')
    df1.drop(['Date'],axis = 1, inplace = True)
    cols = ['vehicle_id','week_start_date',fname]
    df1 = df1[cols]
    return df1

In [ ]:
xx = horsepower_feat(merged_drive_frame, lower=50)

In [ ]:
for i in [50,60,70,80]:
    temp_df = pd.DataFrame()
    if i == 50:
        fin_df_hp = horsepower_feat(merged_drive_frame, lower = i)
    else:
        temp_df = horsepower_feat(merged_drive_frame, lower=i)
        fin_df_hp = fin_df_hp.merge(temp_df, how = 'left', on = ['vehicle_id','week_start_date'])

### RPM Features

In [71]:
merged_drive_frame['rpm_util'] = merged_drive_frame['rpm']/merged_drive_frame['max_horsepower_rpm'] 

In [72]:
def rpm_feat(df, lower=50):
    df = df.copy()
    m1 = (df['rpm_util'] >= lower/100) & (df['rpm_util'] < (lower + 10)/100)
    fname = 'ft_rpm_util_' + str(lower) + 'pct_s'
    df[fname] = np.where(m1,1,0)
    df['datetime'] = pd.to_datetime(df['datetime']) - pd.to_timedelta(7, unit='h')
    df['Date'] = df['datetime'] - pd.to_timedelta(7, unit='d')
    df1 = df.groupby(['vehicle_id', pd.Grouper(key='Date', freq='W-SUN')])[fname].sum().reset_index().sort_values('Date')
    df1['week_start_date'] = df1['Date'] + pd.to_timedelta(1, unit='d')
    df1.drop(['Date'],axis = 1, inplace = True)
    cols = ['vehicle_id','week_start_date',fname]
    df1 = df1[cols]
    return df1

In [73]:
for i in [50,60]:
    temp_df = pd.DataFrame()
    if i == 50:
        fin_df_rpm = rpm_feat(merged_drive_frame, lower = i)
    else:
        temp_df = rpm_feat(merged_drive_frame, lower=i)
        fin_df_rpm = fin_df_hp.merge(temp_df, how = 'left', on = ['vehicle_id','week_start_date'])

In [74]:
fin_df_rpm[fin_df_rpm['vehicle_id'] == 1000500]

,vehicle_id,week_start_date,ft_horsepower_util_50pct_s,ft_horsepower_util_60pct_s,ft_horsepower_util_70pct_s,ft_horsepower_util_80pct_s,ft_rpm_util_60pct_s
0,1000500,2016-12-26,0,0,0,0,0
23,1000500,2017-01-02,0,0,0,0,0
40,1000500,2017-01-09,0,0,0,0,0
61,1000500,2017-01-16,0,0,0,0,0
86,1000500,2017-01-23,0,0,0,0,0
109,1000500,2017-01-30,0,0,0,0,0
121,1000500,2017-02-06,0,0,0,0,0
144,1000500,2017-02-13,0,0,0,0,0
159,1000500,2017-02-20,0,0,0,0,0
180,1000500,2017-02-27,0,0,0,0,0


## Drive Features

In [50]:
path = r'/Users/ankit/Desktop/mck_qb_test/Hackathon/Input data/trip' # use your path
all_files = glob.glob(path + "/*.parquet")

li = []

for filename in all_files:
    df = pd.read_parquet(filename)
    li.append(df)

trip_frame = pd.concat(li, axis=0, ignore_index=True)

In [51]:
trip_frame['datetime'] = pd.to_datetime(trip_frame['datetime']) - pd.to_timedelta(7, unit='h')

In [52]:
trip_frame[trip_frame['vehicle_id'] == 1000515].to_csv('sample_drive_1000515.csv',index = False)

In [53]:
trip_frame.sort_values(by = ['vehicle_id','datetime'], inplace=True)

In [54]:
trip_frame['velocity_diff'] = trip_frame['velocity'].diff()
trip_frame['vehicle_id_diff'] = trip_frame['vehicle_id'].diff()

In [55]:
trip_frame.head()

,vehicle_id,trip_id,datetime,lat,long,velocity,velocity_diff,vehicle_id_diff
7406337,1000500,4f874888ce404720a203e36f1cf5b716,2017-01-01 11:00:00,31.3125,-85.187500,0.00,NaN,NaN
7406338,1000500,4f874888ce404720a203e36f1cf5b716,2017-01-01 11:00:01,31.3125,-85.187222,61.20,61.20,0.0
7406339,1000500,4f874888ce404720a203e36f1cf5b716,2017-01-01 11:00:02,31.3125,-85.186944,65.41,4.21,0.0
7406340,1000500,4f874888ce404720a203e36f1cf5b716,2017-01-01 11:00:03,31.3125,-85.186667,57.70,-7.71,0.0
7406341,1000500,4f874888ce404720a203e36f1cf5b716,2017-01-01 11:00:04,31.3125,-85.186389,68.61,10.91,0.0


In [56]:
trip_frame[trip_frame['vehicle_id_diff'] == 1]['velocity_diff'] = 0

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [57]:
m1 = trip_frame['velocity_diff'] <0
fname = 'deacc_flag'
trip_frame[fname] = np.where(m1,1,0)

In [58]:
trip_frame['deacc_flag_diff'] = trip_frame['deacc_flag'].diff()

In [59]:
m2 = trip_frame['deacc_flag_diff'] == 1
trip_frame['deacc_final_flag']= np.where(m2,1,0)

In [60]:
trip_frame.groupby(['trip_id'])['deacc_final_flag'].sum()

trip_id
00922df3be5a4589ab385d0c2da2dd81     897
00dc31fe55e24d14989c89de4b3b683b    1217
0156d21e316d4d8b9d5bf6ccff797bf7    2224
01b8a24510cd4e4684d67b96369286e0      28
01c2a70c25e5428bb33811ca5eb19270    2723
01d4a4efe7d14d11a5f9fa5f40bd8bc3      99
0244f7bc747b41fba6fcd75444736621     185
025e2f194863461d9c27b2242007ac09    1528
02abe724b7a943cdae50c69e4f03cc26    1285
02c51e56cc484711b218d3d01196687a    2125
02cdeb03041b450d83b18623ba7d6ae8    4287
0324ddb398fd4e97b73375abcda402b8    1195
032d48f64d4e44af80f1a30ae48f40c6    3002
033a252fccc3488da8f0c343fb93ab10    1146
035f294bfbdd46c290fa023054add699     697
0379dabd796440b1aa11378795e859ed    2936
039c14a9a7df4096bb6e9445136fdf77    2408
03b8ac1525474754b4d89d2d647aa8e4    2390
03c33d016a2e4eb3857b900db61feb0e     190
03e0a7f2f6204a3aa6d4e5ebd2fec5f7    2184
03fd5dc0e51443e69803aa0813f6ccfe    1920
047633791fb243ff814b1b760a7bd5a4    1694
0479672ff07c4fefb7ce001b85435d9c    2994
04a0c61b0db5478e8e8c6052b300a303    2119
04c6db23

In [29]:
trip_frame[trip_frame['trip_id'] == '00922df3be5a4589ab385d0c2da2dd81'].to_csv('trip_00922df3be5a4589ab385d0c2da2dd81.csv',index = False)

In [12]:
import pandas as pd
import glob

path = r'C:/Users/sreekar chidurala/Desktop/july work/quantum black AV/weather' # use your path
all_files = glob.glob(path + "/*.parquet")
li = []
for filename in all_files:
    df = pd.read_parquet(filename)
    li.append(df)

weather_frame = pd.concat(li, axis=0, ignore_index=True)

In [13]:
weather_frame

,x,y,date,time,lat,lon,temperature_data,temperature_unit,precipitation_data,precipitation_unit,wind_ew_data,wind_ew_unit,wind_ns_data,wind_ns_unit
0,85,55,2017-02-15,11Z,31.9375,-114.3125,286.430000,k,0.000000,kg/m^2,1.620000,m/s,-2.720000,m/s
1,191,38,2017-02-23,21Z,29.8125,-101.0625,304.960000,k,0.000000,kg/m^2,4.360000,m/s,0.580000,m/s
2,258,60,2017-02-27,16Z,32.5625,-92.6875,287.590000,k,2.450000,kg/m^2,-3.910000,m/s,-1.110000,m/s
3,113,55,2017-01-25,07Z,31.9375,-110.8125,277.800000,k,0.000000,kg/m^2,2.400000,m/s,-1.440000,m/s
4,304,55,2017-02-03,21Z,31.9375,-86.9375,286.010000,k,0.000000,kg/m^2,2.050000,m/s,-5.160000,m/s
5,223,48,2017-01-08,03Z,31.0625,-97.0625,270.610000,k,0.000000,kg/m^2,1.120000,m/s,-1.310000,m/s
6,101,48,2017-02-18,09Z,31.0625,-112.3125,286.900000,k,0.000000,kg/m^2,-3.910000,m/s,9.950000,m/s
7,130,52,2017-02-13,03Z,31.5625,-108.6875,285.080000,k,1.589200,kg/m^2,-7.620000,m/s,2.780000,m/s
8,308,59,2017-01-12,02Z,32.4375,-86.4375,290.040000,k,0.000000,kg/m^2,-0.590000,m/s,5.010000,m/s
9,120,33,2017-01-12,09Z,29.1875,-109.9375,283.070000,k,0.000000,kg/m^2,-2.040000,m/s,-0.430000,m/s


In [ ]:
weather_frame['datetime'] = pd.to_datetime(trip_frame['datetime']) - pd.to_timedelta(7, unit='h')

In [74]:
trip_frame[(trip_frame['lat'] == -83.6875) & (trip_frame['lat'] == -294.16)]

,vehicle_id,trip_id,datetime,lat,long,velocity,velocity_diff,vehicle_id_diff,deacc_flag,deacc_flag_diff,deacc_final_flag


In [75]:
trip_frame

,vehicle_id,trip_id,datetime,lat,long,velocity,velocity_diff,vehicle_id_diff,deacc_flag,deacc_flag_diff,deacc_final_flag
7406337,1000500,4f874888ce404720a203e36f1cf5b716,2017-01-01 11:00:00,31.3125,-85.187500,0.00,NaN,NaN,0,NaN,0
7406338,1000500,4f874888ce404720a203e36f1cf5b716,2017-01-01 11:00:01,31.3125,-85.187222,61.20,61.20,0.0,0,0.0,0
7406339,1000500,4f874888ce404720a203e36f1cf5b716,2017-01-01 11:00:02,31.3125,-85.186944,65.41,4.21,0.0,0,0.0,0
7406340,1000500,4f874888ce404720a203e36f1cf5b716,2017-01-01 11:00:03,31.3125,-85.186667,57.70,-7.71,0.0,1,1.0,1
7406341,1000500,4f874888ce404720a203e36f1cf5b716,2017-01-01 11:00:04,31.3125,-85.186389,68.61,10.91,0.0,0,-1.0,0
7406342,1000500,4f874888ce404720a203e36f1cf5b716,2017-01-01 11:00:05,31.3125,-85.186111,50.79,-17.82,0.0,1,1.0,1
7406343,1000500,4f874888ce404720a203e36f1cf5b716,2017-01-01 11:00:06,31.3125,-85.185833,72.42,21.63,0.0,0,-1.0,0
7406344,1000500,4f874888ce404720a203e36f1cf5b716,2017-01-01 11:00:07,31.3125,-85.185556,67.09,-5.33,0.0,1,1.0,1
7406345,1000500,4f874888ce404720a203e36f1cf5b716,2017-01-01 11:00:08,31.3125,-85.185278,57.79,-9.30,0.0,1,0.0,0
7406346,1000500,4f874888ce404720a203e36f1cf5b716,2017-01-01 11:00:09,31.3125,-85.185000,63.84,6.05,0.0,0,-1.0,0


In [76]:
temp_w = weather_frame[['lat','lon']].drop_duplicates()

In [78]:
temp_t = trip_frame[['lat','long']].drop_duplicates()

In [79]:
temp_t.columns = ['lat','lon']

In [81]:
ss = pd.concat([temp_w, temp_t])

In [83]:
ss.drop_duplicates().shape

(3032997, 2)

In [84]:
ss.shape

(3037154, 2)

In [ ]:
df['variable'].apply(lambda l: " ".join(l))

In [85]:
l = ["a", "b", "c"]
" ".join(l)

'a b c'